In [ ]:
import torch
import utils
from IPython.display import clear_output
from bokeh.io import show, output_notebook
output_notebook()

Let's do some machine learning! In this exercise, you'll get to play around with a simple 1d regression task, and get a feel for the concepts of function spaces, parameters, datasets, optimization, and generalization.

One thing to note is we will *not* be using neural networks! The reason for this is that neural networks are famously difficult to interpret and understand. To explore these ideas, we'll instead be using a function space called a linear basis, which is simple enough to help develop intuitions.

### Part 1: The Function Space

The function space we will be working with today has eight parameters. The code below will create an interactive plot where you can adjust the parameters in order to pick a function. The format of the plot should hopefully be familiar: the input is a single number, shown on the x-axis, and the output is also a single number, shown on the y-axis.

Try out various parameters and see what different functions result! Some things to try:
 - all parameters are zeros
 - one parameter is 1, the rest are zero (try this for each different parameter)
 - all parameters are non-zero
 - one parameter is large, the others are small

In [ ]:
### Change the parameters using the sliders to choose a function!
show(utils.interactive_plot())

### Part 2: Target Functions

Now that we have a sense for how our function space works, let's try to use it to find a specific function. Let's call the function we are trying to find our "target function". In the code below, there are several target functions, of varying difficulty. For each one, try to guess what the correct parameters are by adjusting your guess until you match it! If you can't match it perfectly, try to get an *approximate* match, where you get the big trends right but maybe miss some of the smaller wiggles.

This process should give you a sense for how a machine learning algorithm works: just like you, a machine learning algorithm is just adjusting parameters until it finds a good match.

(By the way, don't worry if you can't get the harder ones! Just give it a try and move on.)

In [ ]:
from utils import easy_target_1, \
                  easy_target_2, \
                  easy_target_3, \
                  easy_target_4, \
                  medium_target_1, \
                  medium_target_2, \
                  medium_target_3, \
                  medium_target_4, \
                  hard_target_1, \
                  hard_target_2, \
                  hard_target_3, \
                  hard_target_4

### Change this to choose a task
target_function = hard_target_3

show(utils.interactive_plot(target_function=target_function))

### Part 3: Datasets

The exercise we just did was actually a little too easier than what machine learning algorithms have to do. That's because in general, machine learning algorithms won't have access to the entire target function: only a limited dataset. In this section, try to find the target function just like before; but this time, do it while only looking at a dataset.

When you are ready to see whether you got the answer right, toggle the target function and see whether your guess matches the true target. Don't reveal the answer until you are ready!

While solving these puzzles, ask yourself the following questions:
 - Was I able to fit the dataset?
 - Did I find the correct target function? Was I close?
 
(Once again, don't worry about solving everything perfectly, just get a sense for it and move on!)

In [ ]:
from utils import easy_target_small_dataset,  \
                  easy_target_medium_dataset, \
                  easy_target_large_dataset,  \
                  medium_target_small_dataset,  \
                  medium_target_medium_dataset, \
                  medium_target_large_dataset,  \
                  hard_target_small_dataset,  \
                  hard_target_medium_dataset, \
                  hard_target_large_dataset

task = easy_target_medium_dataset # change this line to choose a task
dataset = task.dataset
target_function = task.target_function

show(utils.interactive_plot(target_function=target_function, train_points=dataset))


### Part 4: Gradient Descent

Now that you have an appreciation for the effort it takes to manually search a function space by tweaking parameters, it's time for the moment you've all been waiting for...it's time to make the computer do it!

The search algorithm we are going to use is called gradient descent. It works by taking many small steps in function-space, each time adjusting the parameters to get a slightly better function. Each of these steps is taken in the direction of "best local improvement", called the **gradient**. To compute the gradient, we need to use calculus, but for this exercise, you don't have to worry about it; we can use a Python library called `torch` to do it automatically.

One other thing that we need is a definition of "better function". You and I can just look at a picture to see whether a function fits data points well, but how can we express that mathematically? The intuition we want to capture is that our function should be *close* to the datapoints, so, for any pair `[x,target_function(x)]` in our dataset, `my_function(x)` should be close to `target_function(x)`. One way to measure the distance between two numbers is to take their difference and square it. (Question to ponder: what would happen if we didn't square it?)

Concretely: we want to find the parameters of the function whose average distance from the datapoints is as small as possible. Since we "win" when this is small, it's called the **loss**. The gradient of the loss (with respect to the parameters) tells us how to change the parameters to best *increase* the loss, so we want to update the parameters in the *opposite* direction of the gradient.

Let's see if we can code it up!

In [ ]:
## First, let's pick a random target.
##    (Important: this is a "secret" function! In a real task, we wouldn't have access to this.
##     Our learning algorithm isn't allowed to use it at all, or it would be cheating.)
target_function = utils.get_random_target()

## Next, let's get some examples of our function in action.
## This is what our algorithm is allowed to look at.
dataset = utils.get_examples(target_function, n_datapoints=50) ## once your implementation is done, try playing with this number!

## Now, let's initialize our parameters.
## Since we are going to be searching the space with gradient descent, it doesn't matter too much where we start!
## Feel free to play around with these values.
parameters = [0., 0., 0., 0., 0., 0., 0., 0.]
parameters = torch.nn.Parameter(torch.tensor(parameters)) ## this line just sets up some stuff in the background so that we can take the gradient later

## How long to run our gradient descent process for. The longer we run it, the harder we've searched.
TOTAL_STEPS = 2000
## How big of a step to take. (We multiply the gradient by the learning rate before taking a step.)
## Smaller steps take longer to learn, but if the stepsize is too big, gradient descent breaks and the whole thing goes haywire.
## (You'll learn why gradient descent needs to take tiny steps if you study calculus!)
LEARNING_RATE = .01

## Let's set up our plot so we can watch the learning happen.
plot, my_function_datasource, sliders = utils.live_plot(dataset, utils.function_from_parameters(parameters), target_function)
show(plot, notebook_handle=True)

## Start the main loop of training.
for step_i in range(TOTAL_STEPS):
    ## The current parameters define a function.
    my_function = utils.function_from_parameters(parameters)
    
    ## Update the plot to reflect our current guess.
    utils.update_live_plot(parameters, my_function_datasource, sliders)
    print(f"Step {step_i+1} / {TOTAL_STEPS} ({100*(step_i+1)/TOTAL_STEPS:3}%)", end='\r')
        
    ## Compute the loss on the dataset.
    losses_list = [(my_function(x) - y)**2. for x, y in dataset]
    average_loss = sum(losses_list)/len(losses_list)
    
    ## Compute the gradient from the loss using torch.autograd.
    ## The gradient is a list of 8 numbers, corresponding to each of our 8 parameters.
    ## Each number in the gradient list tells us how to change its corresponding parameter.
    gradient = torch.autograd.grad(average_loss, parameters)[0]
    
    ## Update the parameters using the gradient.
    ##    Note that there is a little trick here. parameters is a list of numbers, and
    ##    gradient is also a list of numbers. But since they are both wrapped in torch.tensor(),
    ##    they behave differently from normal Python lists. One cool thing we can do is "elementwise" math.
    ##    Multiplying a torch.tensor is the same as multiplying *each element* in the torch.tensor,
    ##    and subtracting two torch.tensor objects (of the same length) is equivalent to subtracting
    ##    each element of the second from its corresponding element in the first.
    parameters = parameters - LEARNING_RATE*gradient

And there you go -- your first machine learning algorithm! Hopefully you now understand what this algorithm is doing, and why. Also, hopefully you now have an appreciation for just how cool it is that gradient descent is able to search a big function space so quickly!

Of course, this is only the beginning. 1d regression is nice to visualize, but kind of boring. Things get *really* cool when we think about functions which operate on high-dimensional spaces in complex ways. For example, there's a function which maps from chessboard positions into optimal chess moves -- if we can learn that one, we can beat anyone at chess! Or the function that maps from an image of a person's face to an image of that face with a dog-nose on it, like SnapChat does with their filters. The possibilities are seemingly endless.

Amazingly, to learn all these really cool functions (and more), we can use almost the *exact same algorithm* as we used here! The main difference is we switch from the simple 8-parameter function space we used today, for a **deep neural network** function space, which has millions or even billions of parameters. Obviously, we could never have searched such a space by hand! But by using gradient descent, which we learned about today, we are able to do so.